# Análise ENEM 2020

## Carregando Bibliotecas

### Variável de ambiente

In [1]:
import os

os.environ["SPARK_HOME"] = "D:\spark\spark-3.2.1-bin-hadoop3.2"
os.environ['HADOOP_HOME'] = "D:\spark\spark-3.2.1-bin-hadoop3.2\hadoop" 

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

### Pyspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

import pyspark.sql as py

import pyspark.pandas as ps

In [4]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

### Visualização

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

## Carregando os dados

In [6]:
enem_2020 = ps.read_csv("base_dados/enem/microdados_enem2020/DADOS/MICRODADOS_ENEM_2020.csv", sep=';')

In [7]:
columns=['TP_FAIXA_ETARIA', 'TP_COR_RACA', 'TP_ESTADO_CIVIL' ,'TP_SEXO', 'TP_ST_CONCLUSAO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006']

In [9]:
enem_2020_selected = enem_2020[columns]

## Verificando Valores Ausentes

### Visualização

In [10]:
import plotly.graph_objects as go

In [11]:
missing = enem_2020_selected.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace = True)

In [12]:
texto_grafico = []
for i in range(len(missing)):
    texto_grafico.append(str(missing.values[i]))

D:\spark\spark-3.2.1-bin-hadoop3.2\python\pyspark\pandas\generic.py:638: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))


In [13]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=missing.index.tolist(),
    y=missing.values,
    #mode="markers+text",
    name="Markers and Text",
    text=texto_grafico,
    textposition="outside"
))

fig.update_layout(
    autosize=False,
    width=1000,
    height=500)

fig.update_layout(title_text="Quantidade de Dados Ausentes por Colunas (NAN)",
                  title_font_size=20)

fig.update_xaxes(title_text="Colunas")
fig.update_yaxes(title_text="Quantidade de Valores Ausentes")

fig.show()

D:\spark\spark-3.2.1-bin-hadoop3.2\python\pyspark\pandas\generic.py:638: UserWarning:

We recommend using `Series.to_numpy()` instead.



### Tabela com Porcentagem

In [14]:
var_with_nan = []
def verfify_nan_values(df):
    var_without_nan = []
    var_column = []
    var_percent = []
    total = df.shape[0]
    count = 0
    for index, value in df.isna().sum().iteritems():
        if value == 0:
            var_without_nan.append(index)
        else:
            percent_with_nan = (value / total) * 100
            var_with_nan.append(count)
            var_column.append(index)
            var_percent.append(percent_with_nan)
        count  = count + 1

    d = {'coluna': var_column, 'porcentagem_nan %': var_percent}
    df_nan = ps.DataFrame(data=d)

    return df_nan

In [15]:
df_nan = verfify_nan_values(enem_2020_selected)

df_nan

,coluna,porcentagem_nan %
0,NU_NOTA_CN,55.085751
1,NU_NOTA_CH,52.376135
2,NU_NOTA_LC,52.376135
3,NU_NOTA_MT,55.085751
4,NU_NOTA_REDACAO,52.376135
5,Q001,1.659419
6,Q002,1.659419
7,Q003,1.659419
8,Q004,1.659419
9,Q005,1.659419


### Tratamento dos Dados Ausentes

In [16]:
enem_2020_without_nan = enem_2020_selected.dropna(subset = ['NU_NOTA_CN', 'NU_NOTA_CH', 'Q001'])

## Adicionando Média das Notas

Pesquisa com Base curso de Medicina na USP

https://www.sisu.pro.br/sisu/sisu-2021-usp-2.html#:~:text=Pesos%20para%20Medicina&text=Linguagens%20e%20C%C3%B3digos%3A%202%C3%97,Matem%C3%A1tica%3A%203%C3%97500%20%3D%201.500

* Redação: 2
* Linguagens e Códigos: 2
* Ciências Humanas: 2
* Ciências da Natureza: 4
* Matemática: 3

In [17]:
def mean_med(df):
    df['MEDIA_MED'] = ( 
        df['NU_NOTA_CN'] * 4 + 
        df['NU_NOTA_CH'] * 2 + 
        df['NU_NOTA_LC'] * 2 + 
        df['NU_NOTA_MT'] * 3 + 
        df['NU_NOTA_REDACAO'] * 2
    ) / (4+2+2+3+2)

In [18]:
mean_med(enem_2020_without_nan)

Pesquisa com Base curso de Administração na Unicentro Paraná

https://www3.unicentro.br/vestibular/sisu/cursos-e-pesos/


* Redação: 1
* Linguagens e Códigos: 2
* Ciências Humanas: 2
* Ciências da Natureza: 2
* Matemática: 3

In [19]:
def mean_adm(df):
    df['MEDIA_ADM'] = ( 
        df['NU_NOTA_CN'] * 2 + 
        df['NU_NOTA_CH'] * 2 + 
        df['NU_NOTA_LC'] * 2 + 
        df['NU_NOTA_MT'] * 3 + 
        df['NU_NOTA_REDACAO'] * 1
    ) / (2+2+2+3+1)

In [20]:
mean_adm(enem_2020_without_nan)

## Análise dos Agrupamentos

In [21]:
enem_2020_without_nan.columns

Index(['NU_IDADE', 'TP_COR_RACA', 'TP_ESTADO_CIVIL', 'TP_SEXO',
       'TP_ST_CONCLUSAO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005',
       'Q006', 'MEDIA_MED', 'MEDIA_ADM'],
      dtype='object')

#### Media por Raça

In [22]:
enem_group_raca_media = enem_2020_without_nan[['TP_COR_RACA', 'MEDIA_MED', 'MEDIA_ADM']].groupby(['TP_COR_RACA']).mean()

In [23]:
enem_group_raca_media = enem_group_raca_media.reset_index()

In [24]:
legenda_raca = {
0 : 'Não declarado',
1 : 'Branca',
2 : 'Preta',
3 : 'Parda',
4 : 'Amarela',
5 : 'Indígena',
6 : 'Não dispõe da informação'
}

In [25]:
enem_group_raca_media['TP_COR_RACA'] = enem_group_raca_media['TP_COR_RACA'].map(legenda_raca)

In [26]:
enem_group_raca_media.sort_values(by=['MEDIA_MED'])

,TP_COR_RACA,MEDIA_MED,MEDIA_ADM
2,Indígena,468.917145,469.380467
4,Preta,495.053620,495.231243
1,Parda,502.918815,502.675000
3,Amarela,519.638740,519.290911
5,Não declarado,528.470517,528.840026
0,Branca,550.124117,550.145222


#### Media por Renda

In [45]:
legenda_renda = {
'A':'Nenhuma Renda',
'B':'Até R$ 1.045,00',
'C':'De R$ 1.045,01 até R$ 1.567,50',
'D':'De R$ 1.567,51 até R$ 2.090,00',
'E':'De R$ 2.090,01 até R$ 2.612,50',
'F':'De R$ 2.612,51 até R$ 3.135,00',
'G':'De R$ 3.135,01 até R$ 4.180,00',
'H':'De R$ 4.180,01 até R$ 5.225,00',
'I':'De R$ 5.225,01 até R$ 6.270,00',
'J':'De R$ 6.270,01 até R$ 7.315,00',
'K':'De R$ 7.315,01 até R$ 8.360,00',
'L':'De R$ 8.360,01 até R$ 9.405,00',
'M':'De R$ 9.405,01 até R$ 10.450,00',
'N':'De R$ 10.450,01 até R$ 12.540,00',
'O':'De R$ 12.540,01 até R$ 15.675,00',
'P':'De R$ 15.675,01 até R$ 20.900,00',
'Q':'Acima de R$ 20.900,00'
}

In [40]:
enem_group_renda_media = enem_2020_without_nan[['Q006', 'MEDIA_MED', 'MEDIA_ADM']].groupby(['Q006']).mean()

In [42]:
enem_group_renda_media = enem_group_renda_media.reset_index()

In [47]:
enem_group_renda_media['Q006'] = enem_group_renda_media['Q006'].map(legenda_renda)

In [49]:
enem_group_renda_media.rename(columns={'Q006': 'Faixa Renda'}, inplace=True)

In [48]:
enem_group_renda_media.sort_values(by=['MEDIA_MED'])

,Q006,MEDIA_MED,MEDIA_ADM
11,Nenhuma Renda,465.499549,465.419109
4,"Até R$ 1.045,00",481.967149,481.647560
9,"De R$ 1.045,01 até R$ 1.567,50",505.112431,505.409640
7,"De R$ 1.567,51 até R$ 2.090,00",519.198730,519.609885
3,"De R$ 2.090,01 até R$ 2.612,50",535.807729,536.251117
1,"De R$ 2.612,51 até R$ 3.135,00",545.886771,545.904329
13,"De R$ 3.135,01 até R$ 4.180,00",562.498036,562.365734
16,"De R$ 4.180,01 até R$ 5.225,00",576.821714,576.370507
14,"De R$ 5.225,01 até R$ 6.270,00",587.379963,586.665846
10,"De R$ 6.270,01 até R$ 7.315,00",595.391831,594.571242


#### Media por Renda e por Raça

In [58]:
enem_group_renda_raca_media = enem_2020_without_nan[['Q006', 'TP_COR_RACA', 'MEDIA_MED', 'MEDIA_ADM']].groupby(['Q006', 'TP_COR_RACA']).mean()

In [59]:
enem_group_renda_raca_media = enem_group_renda_raca_media.reset_index()

In [60]:
enem_group_renda_raca_media['Q006'] = enem_group_renda_raca_media['Q006'].map(legenda_renda)
enem_group_renda_raca_media['TP_COR_RACA'] = enem_group_renda_raca_media['TP_COR_RACA'].map(legenda_raca)

In [61]:
enem_group_renda_raca_media.rename(columns={'Q006': 'Faixa Renda'}, inplace=True)

In [62]:
enem_group_renda_raca_media.sort_values(by=['MEDIA_MED'])

,Faixa Renda,TP_COR_RACA,MEDIA_MED,MEDIA_ADM
29,Nenhuma Renda,Indígena,436.855819,438.297140
8,"Até R$ 1.045,00",Indígena,456.439396,456.697376
52,Nenhuma Renda,Preta,460.002618,460.122666
26,Nenhuma Renda,Parda,460.754443,460.498184
74,Nenhuma Renda,Amarela,463.313031,462.840133
38,Nenhuma Renda,Não declarado,470.218311,470.847215
93,"Até R$ 1.045,00",Preta,475.087654,474.923336
69,"Até R$ 1.045,00",Parda,477.444312,476.995359
99,"De R$ 1.045,01 até R$ 1.567,50",Indígena,477.837504,478.758276
77,"Até R$ 1.045,00",Amarela,480.071985,479.300768


#### Quantidade por Raça

In [63]:
enem_group_raca_quant = enem_2020_without_nan[['TP_COR_RACA', 'MEDIA_MED']].groupby(['TP_COR_RACA']).count()

In [64]:
enem_group_raca_quant = enem_group_raca_quant.reset_index()

In [65]:
enem_group_raca_quant['TP_COR_RACA'] = enem_group_raca_quant['TP_COR_RACA'].map(legenda_raca)

In [66]:
enem_group_raca_quant.sort_values(by=['MEDIA_MED'])

,TP_COR_RACA,MEDIA_MED
2,Indígena,13820
5,Não declarado,49395
3,Amarela,56786
4,Preta,309833
0,Branca,974067
1,Parda,1157403


#### Quantidade por Renda e Raça

In [67]:
enem_group_raca_renda_quant = enem_2020_without_nan[['TP_COR_RACA', 'Q006', 'MEDIA_MED']].groupby(['TP_COR_RACA', 'Q006']).count()

In [70]:
enem_group_raca_renda_quant = enem_group_raca_renda_quant.reset_index()

In [71]:
enem_group_raca_renda_quant['TP_COR_RACA'] = enem_group_raca_renda_quant['TP_COR_RACA'].map(legenda_raca)
enem_group_raca_renda_quant['Q006'] = enem_group_raca_renda_quant['Q006'].map(legenda_renda)

In [72]:
enem_group_raca_renda_quant.sort_values(by=['MEDIA_MED'])

,TP_COR_RACA,Q006,MEDIA_MED
46,Indígena,"Acima de R$ 20.900,00",20
77,Indígena,"De R$ 12.540,01 até R$ 15.675,00",27
0,Indígena,"De R$ 15.675,01 até R$ 20.900,00",28
13,Indígena,"De R$ 8.360,01 até R$ 9.405,00",28
90,Indígena,"De R$ 7.315,01 até R$ 8.360,00",32
69,Indígena,"De R$ 9.405,01 até R$ 10.450,00",36
75,Indígena,"De R$ 10.450,01 até R$ 12.540,00",40
17,Indígena,"De R$ 6.270,01 até R$ 7.315,00",59
33,Indígena,"De R$ 5.225,01 até R$ 6.270,00",100
4,Indígena,"De R$ 4.180,01 até R$ 5.225,00",199
